**Andere manier om te kijken naar evaluation, i.p.v terug te splitten en te predicten, nu opgeslagen predicition gebruiken**

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Path to the saved model directory
save_path = r"C:\Users\corne\OneDrive - KU Leuven\Thesis\Working Code\SAVED-Models\GroNLP\Run_2025-04-23_18-28"

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained(save_path)
model = BertForSequenceClassification.from_pretrained(save_path)
model.eval()  # Set to eval mode

# Optional: Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30073, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F
import os
import json  # Needed for loading the mappings

# === Define path to the saved run ===
save_path = r"C:\Users\corne\OneDrive - KU Leuven\Thesis\Working Code\SAVED-Models\GroNLP\Run_2025-04-23_18-28"

# ✅ Load the saved test predictions 
df = pd.read_csv(os.path.join(save_path, "test_predictions.csv"))

# ✅ Recreate logits tensor from the CSV
logits = torch.tensor(df["logits"].apply(eval).tolist())

# ✅ Apply softmax to get prediction probabilities
probabilities = F.softmax(logits, dim=1)

# ✅ Extract raw values
texts = df["text"].tolist()
true_labels_ids = df["true_label"].tolist()
predicted_label_ids = df["predicted_label"].tolist()

# ✅ Convert label IDs to themes using the mappings
with open(os.path.join(save_path, "label_mappings.json"), "r", encoding="utf-8") as f:
    mappings = json.load(f)

theme_to_id = mappings["theme_to_id"]
id_to_theme = {int(k): v for k, v in mappings["id_to_theme"].items()}  # convert keys back to int
unique_themes = mappings["unique_themes"]  # Load the unique themes list


In [5]:
# Filter confidence range first
wrong_pool = data[
    (data["correct"] == False) &
    (data["confidence"] >= 0.60) &
    (data["confidence"] < 0.99)
]

right_pool = data[
    (data["correct"] == True) &
    (data["confidence"] >= 0.60) &
    (data["confidence"] < 0.99)
]

# Sample randomly from filtered sets
wrong_filtered = wrong_pool.sample(n=5, random_state=42)
right_filtered = right_pool.sample(n=5, random_state=24)

# Display for confirmation
print("❌ Random Misclassified Examples (confidence 0.60–0.99):")
for i, row in wrong_filtered.iterrows():
    print(f"\nID {i} | True: {row['true_theme']} | Pred: {row['pred_theme']} | Conf: {row['confidence']:.2f}")
    print(f"→ {row['text'][:300]}")

print("\n✅ Random Correctly Classified Examples (confidence 0.60–0.99):")
for i, row in right_filtered.iterrows():
    print(f"\nID {i} | True: {row['true_theme']} | Pred: {row['pred_theme']} | Conf: {row['confidence']:.2f}")
    print(f"→ {row['text'][:300]}")


❌ Random Misclassified Examples (confidence 0.60–0.99):

ID 6717 | True: Cultuur en Communicatie | Pred: Onderwijs en Samenleving | Conf: 0.99
→ Uit hoeveel mensen zal de afdeling Digitalisering bestaan, wie zal ze leiden en wanneer moet ze operationeel zijn?

ID 5229 | True: Toerisme | Pred: Bestuur en Beleid | Conf: 0.60
→ Hoeveel van de klachten overgemaakt aan de Vlaamse Ombudsdienst werden ontvankelijk verklaard?

ID 9018 | True: Bestuur en Beleid | Pred: Milieu en Landbouw | Conf: 0.83
→ Hoe apprecieert de minister die cijfers? Zo ja, wat zijn volgens de minister daarvan de oorzaken?

ID 1022 | True: Milieu en Landbouw | Pred: Welzijn en Gezondheid | Conf: 0.62
→ Heeft de minister daarover een evaluatie gemaakt? Zo ja, wat waren de bevindingen van de minister?

ID 5784 | True: Milieu en Landbouw | Pred: Mobiliteit en Infrastructuur | Conf: 0.96
→ Kan zij per actie het volgende specifiëren: naam van de maatregel; doelstelling van de maatregel; verantwoordelijke administratie; stan

In [ ]:
def predict_proba(texts):
    encoded = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = F.softmax(outputs.logits, dim=1)

    return probs.cpu().numpy()


In [9]:

from lime.lime_text import LimeTextExplainer
import os

# Setup LIME explainer
class_names = [id_to_theme[i] for i in sorted(id_to_theme.keys())]
explainer = LimeTextExplainer(class_names=class_names)

# Folder to save explanations
lime_output_dir = "lime_explanations"
os.makedirs(lime_output_dir, exist_ok=True)

# Combine both sets
all_examples = pd.concat([
    wrong_filtered.assign(type="wrong"),
    right_filtered.assign(type="correct")
])
# === Specify the ID of the example you want to explain ===
example_id = 5229  # 👈 change this to whatever ID you want

# === Fetch the row from the full DataFrame (e.g., `data` or `all_examples`) ===
row = data.loc[example_id]  # or all_examples.loc[example_id] if you're using a filtered subset

sample_text = row["text"]
text_snippet = sample_text[:300].replace("\n", " ").strip()
true_theme = row["true_theme"]
pred_theme = row["pred_theme"]
pred_id = row["pred_label_id"]
conf = row["confidence"]
label_type = "correct" if row["correct"] else "wrong"

print(f"\n🔍 ID {example_id} | {label_type.upper()} prediction")
print(f"→ True: {true_theme}")
print(f"→ Predicted: {pred_theme}")
print(f"→ Confidence: {conf:.2f}")
print(f"→ Text: {text_snippet}")

# Run LIME
explanation = explainer.explain_instance(
    sample_text,
    predict_proba,
    num_features=10,
    labels=[pred_id],
    num_samples=1000
)

# Save to file
safe_pred = pred_theme.replace(" ", "_").replace("/", "-")
safe_true = true_theme.replace(" ", "_").replace("/", "-")
filename = f"{label_type}_id{example_id}_{safe_pred}_vs_{safe_true}.html"
path = os.path.join(lime_output_dir, filename)
explanation.save_to_file(path)

print(f"\n✅ Saved LIME explanation to: {path}")



🔍 ID 5229 | WRONG prediction
→ True: Toerisme
→ Predicted: Bestuur en Beleid
→ Confidence: 0.60
→ Text: Hoeveel van de klachten overgemaakt aan de Vlaamse Ombudsdienst werden ontvankelijk verklaard?

✅ Saved LIME explanation to: lime_explanations\wrong_id5229_Bestuur_en_Beleid_vs_Toerisme.html
